In [1]:
!pip install dadmatools spacy_stanza hazm ipywidgets 

     |████████████████████████████████| 848 kB 24.4 MB/s 
     |████████████████████████████████| 316 kB 36.8 MB/s 
     |████████████████████████████████| 176 kB 60.0 MB/s 
     |████████████████████████████████| 2.6 MB 65.9 MB/s 
     |████████████████████████████████| 12.7 MB 56.5 MB/s 
     |████████████████████████████████| 4.1 MB 53.3 MB/s 
     |████████████████████████████████| 68 kB 6.5 MB/s 
     |████████████████████████████████| 462 kB 65.7 MB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 1.6 MB 43.1 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 5.6 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
     |████████████████████████████████| 131 kB 45.2 MB/s 
     |████████████████████████████████| 57 kB 2.3 MB/s 
     |█████

### Read symbols

In [3]:
import stanza
import spacy_stanza

stanza.install_corenlp()
stanza.download("fa")
stanza_nlp = spacy_stanza.load_pipeline("fa")


2022-04-15 08:20:22 INFO: Installing CoreNLP package into /root/stanza_corenlp...


2022-04-15 08:20:32 INFO: Downloading default packages for language: fa (Persian)...


2022-04-15 08:20:38 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-04-15 08:20:39 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-04-15 08:20:39 INFO: Use device: cpu
2022-04-15 08:20:39 INFO: Loading: tokenize
2022-04-15 08:20:39 INFO: Loading: mwt
2022-04-15 08:20:39 INFO: Loading: pos
2022-04-15 08:20:39 INFO: Loading: lemma
2022-04-15 08:20:39 INFO: Loading: depparse
2022-04-15 08:20:40 INFO: Done loading processors!


In [252]:
import pandas as pd 
import re 

df = pd.read_csv("symbols_info.csv", index_col=0)
df['Corp'] = df['Corp. Title'].apply(lambda word: re.sub(r"\(.*\)|(ح\.)|(ح\s\.)", "", word).strip())
df = df[['Symbol', 'Corp']]
mask = df['Corp'] == df['Symbol']
df['Corp'][mask] = 'شرکت ' + df['Corp']
df.head(n=10)


,Symbol,Corp
0,ثملی,بین المللی ساختمان و صنعت ملی
1,اخابرح,مخابرات ایران
2,شتولیح,تولی‌پرس‌
3,دسبحانح,سبحان دارو
4,وزمین,بانک ایران زمین
5,سفارسح,سیمان فارس و خوزستان
6,فنوالح,نورد آلومینیوم‌
7,غالبرح,لبنیات ‌کالبر
8,خساپاح,سایپا
9,مبین,مبین انرژی خلیج فارس


In [304]:
examples = [
    "برکت امروز اطلاعیه‌ای مهم منتشر میکند.",
    "نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    "نماد برکت امروز عرضه ی اولیه دارد.",
    "عرضه های اولیه امروز خوب هستند.",
    "عرضه اولیه‌های امروز خوب هستند.",
    "این عرضه‌ی اولیه خیلی خوبه",
    "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    "آ س پ امروز روند مثبتی داشت.",
    "آمریکا موجب ریزش بازار شد",
    "ارزش سهام مخابرات ایران امروز کاهش زیادی یافت.‎",


]

texts = [
    # "رشد مثبت بد است",
    # "او یک سیگنال مثبت داد",
    # "نماد برکت افزایش و نماد کگل کاهش یافت.",
    # "برکت امروز اطلاعیه‌ای مهم منتشر میکند.",
    # "نماد برکت امروز عرضه‌ی اولیه خیلی خوبی داره.",
    # "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    # "نماد برکت امروز عرضه ی اولیه دارد.",
    # "نماد برکت امروز عرضه ی اولیه خیلی خوبی داره.",
    # "عرضه های اولیه امروز خوب هستند.",
    # "عرضه اولیه‌های امروز خوب هستند.",
    # "این عرضه‌ی اولیه خیلی خوبه",
    # "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    # "نماد برکت امروز عرضه اولیه خیلی خوبی داره.",
    # 'روز چهارشنبه یک دفعه برای خودشون افشا زدن.',
    # "سهام وغدیر و خزر کاهش یافت.",
    # "برای خودشون ی افشایی زدن.",
    # "آ س پ امروز بالا رفت.",
    # "ریزش بازار به دلیل حمله‌ی روسیه هست.",
    # "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    # "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    # "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",
    # "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    # "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    # "شاخص به ۲ میلیون می‌رسه",
    # "آ س پ امروز روند مثبتی داشت.",
    # "آمریکا باعث ریزش بازار شد",
    # "آمریکا موجب ریزش بازار شد",
    # "آمریکا دلیل ریزش بازار شد",
    # "کاهش قیمت سهم عجیب بود",
    # "قیمت زیاد شد",
    # "قیمت زیاد است",
    # "ارزش سهام مخابرات ایران امروز کاهش زیادی یافت.‎",
    # "ریزش مثبت رشد بازار خوب است",
    # "به کتابخانه رفتم.",
    # "به کتابخانه رفت.",
    # "پول در جیب من است.",
    # "رشد مثبت نوسان بازار",
    # "هفته دیگه صعود خیلی زیاد برکت رو خواهید دید",
    # "شک نکنین امروز بمپنا صف خرید می شه.",
    # "به نظرم کگل فردا میتونه به روند صعودی خودش برگردد",
    # "صعود خیلی زیاد برکت را خواهید دید",
    # "رشد مثبت نوسان بازار",
    "سهم قیمتش پایین است",
    # "حضور تو موجب خوشحالی من در هوای بارانی است",
    # "کاهش قیمت سهم خوب است",
    # "کاهش قیمت سهم",
    # "قیمت سهم کاهش یافت",
    # "کاهش زیادی یافت",
    # "بازار روندی متعادل و مثبت دارد",
    # "بازار ده واحد افت کرد",
    # "شاخص ۱۰ واحد رشد کرد",
    # "روز دوشنبه شاخص بورس به صعود ادامه داد و برای دومین روز متوالی افزایش یافت",
    # "نماد با کاهش بیش از ۳ درصدی قیمت همراه بود",
    # "جلسه‌ای برای افزایش کارایی و افزایش نقد شوندگی بازار سرمایه مقرر شد",
    # "جلسه‌ای برای افزایش کارایی و افزایش نقد شوندگی بازار سرمایه مقرر شد",
    # "شاخص با ده واحد افزایش به سقف خود رسید",
    # "کارشناسان، روند متعادل و گاها نزولی برای بازار سهام امروز پیش‌بینی کردند",
    # "رشد مثبت بازار خوب است",
    # "کارشناسان، روند نزولی و خوب برای بازار سهام امروز پیش‌بینی کردند",  #Problem, because there is ebham in sent
    # "بورس سه شنبه افت بیش از ۳۳ هزار واحدی داشت",
    # "امروز چند سیگنال برای سهام‌داران وجود دارد", #be nazaram vojood darad ham bayad begin
    # "اصلاح و اعمال تدریجی قوانین بورسی کمک کننده است",
    # "شاخص با ده واحد افزایش به سقف خود رسید",
    # "افزایش ۱۰۰ واحدی قیمت سهام رخ داد",
    # "وعده‌ی اصلاح برخی قوانین مزاحم داده شد",
    # "شاخص بورس ۷ هزار واحد افت کرد",
    # "برخی گمانه‌زنی‌ها از روند رو به مثبت جهش بازار سرمایه خبر می‌دهد",
    # "برخی گمانه‌زنی‌ها از روند مثبت بازار خبر می‌دهند",
    # "برخی گمانه‌زنی‌ها از روند مثبت بازار، خبر می‌دهند",
    # "نماد با کاهش بیش از ۳ درصدی قیمت همراه بود",
    # "روند نزولی ملایم خوب است",
    # "شاخص معاملات ۱۱ درصد افت، ۱۲.۵ درصد رشد و ۲.۲ درصد افزایش یافت", #hazf be gharine?
    # "برکت ۱۱ درصد افت، خزر ۱۲.۵ درصد رشد و کگل ۳ درصد رشد پیدا کرد",
    # "شاخص معاملات ۱۱ درصد افت پیدا کرد، ۱۲.۵ درصد رشد کرد و ۲.۲ درصد افزایش یافت",
    # "خروج سرمایه تاثیر مثبتی بر بازار داشت. اما سهام آ س پ سقوط کرد.",
    # "امروز رشد سهام خودرو مثبت شد",
    # "روند مثبت سهام جالب بود",
    # "مثبت بودن رشد سهام موجب سقوط قیمت نفت شد",
    # "بررسی تغییرات مثبت سهام امروز انجام شد",
    # "سهم وغدیر عضو مثبت‌های بازار بورس بود",
    # "علت رکود رشد مثبت بازار سهام است",
    # "علت رکود ۳ درصد رشد مثبت بازار سهام است",
    # "علت رشد مثبت بازار سهام رکود بازار است",
    # "بزرگترین مشکل بازار، ریزش سهام است",
    # "بزرگترین مشکل بازار حضور بازیگر است",
    # "بزرگترین مشکل بازار حضور بازیگر در ایران است",
    # "رشد مثبت بازار و حضور بازیگر باعث ضرر شد",
    # "روند صعودی و متعادل",
    # "روند صعودی و ریزش بازار",
    # "امروز صعود بازار به شدت کاهش زیادی یافت",
    # "امروز عرضه اولیه‌ای بزرگ رخ داد",
    # "امروز خروج پولی رخ داد",
    # "امروز خروج پول ی رخ داد",
    # "امروز خروج پول‌ی رخ داد",
    # "نماد برکت امروز عرضه‌ی اولیه خوبی داره",
    # "نماد برکت امروز عرضه ی اولیه خوبی داره",
    # "نماد برکت امروز عرضه اولیه خوبی داره",
    # "نماد برکت امروز عرضه اولیه‌ای داره",
    # "نماد برکت امروز عرضه اولیه ای خوب داره",
    # "نماد برکت امروز عرضه اولیه‌های خوبی داره",
    # "نماد برکت امروز عرضه اولیه‌ هایی داره",
    # "قیمت سهم افزایش یافت",
    # "گربه از پشت بام افتاد",
    # "مخابرات ایران رشد مثبتی را تجربه کرد",
]

events = [
    "سود",
    "ضرر",
    "مثبت",
    "منفی",
    "صعود",
    "نزول",
    "افزایش",
    "کاهش",
    "رشد",
    "ریزش",
    "افشا",
    "افشا ب",
    "افشا الف",
    "اصلاح",
    "افت",
    "نوسان",
    "نوسانات",
    "سقوط",
    "تقسیم سود",
    "تعیین نرخ",
    "عرضه اولیه",
    "صف خرید",
    "صف فروش",
    "خروج سرمایه",
    "افزایش سرمایه"
    "خروج پول",
    "مجمع عمومی",
]
announcements = [
    "اطلاعیه",
    "گزارش",
    "اعلامیه",
    "اعلانیه"
]
analyse = [
    "سهم رانتی",
    "تحلیل تکنیکال",
    "تحلیل فاندامنتال",
    "کندل استیک",
    "سیگنال",
    "تکنیکال",
    "کندل",
    "تیک",
    "سرشانه",
    "مقاومت",
    "حمایت",
    "کراس",
    "واگرایی",
    "اندیکاتور",
    "سقف",
    "کف",
    "قیبوناچی",
    "فیبوناتچی",
    "فیبو",
    "کانال",
    "کف‌‌سازی",
    "کف سازی",
    "رنج",
]
characters =[
    "بازیگر",
    "حقیقی",
    "حقوقی",
    "بازی‌گردان",
    "نوسان‌گیر",
]

In [281]:
from dadmatools.models.normalizer import Normalizer as N1
from hazm import Normalizer as N2

normalizer1 = N1(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="<URL",
    replace_mobile_number_with="<MOBILE_NUMBER>",
    replace_emoji_with="<EMOJI>",
    replace_home_number_with="<HOME_NUMBER>",
)
normalizer2 = N2()

In [265]:
from spacy.matcher import Matcher
matcher: Matcher = Matcher(stanza_nlp.vocab, validate=True)

def add_patterns(matcher, keywords, class_name):
  keyword_patterns = []
  for keyword in keywords:
      keyword_tokens = stanza_nlp.tokenizer(keyword)

      pre_det = ["این", "هر", "چند", "چندین", "آن", "همین", "همان", "چنین", "چنین"]
      if len(keyword_tokens) == 1:
          token1_keyword = keyword_tokens[0].text
          r = '(ی|ای|یی|ها|هایی|های)?'
          pattern = [
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": rf'\b{token1_keyword}{r}\b'}},
              {'TEXT': {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},
          ]

      elif len(keyword_tokens) == 2:
          token1_keyword = keyword_tokens[0].text
          token2_keyword = keyword_tokens[1].text
          pattern = [
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": token1_keyword}},
              {'TEXT': {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},
              {'TEXT': {"IN": pre_det}, "OP": "?"},
              {'TEXT': {"REGEX": token2_keyword}},
              {'TEXT': {"IN": ["ای", "ی", "ها", "های", "هایی"]}, "OP": "?"},      #TODO: This is not working
          ]
          
      keyword_patterns.append(pattern)

  matcher.add(class_name, keyword_patterns, greedy="LONGEST")

SYMBOL = "SYMBOL"
CORP = "CORP"
EVENT = "EVENT"
ANALYSE = "ANALYSE"
ANNOUNCE = "ANNOUNCE"
CHARACTERS = "CHARACTERS"

add_patterns(matcher, events, "EVENT")
add_patterns(matcher, analyse, "ANALYSE")
add_patterns(matcher, announcements, "ANNOUNCE")
add_patterns(matcher, characters, "CHARACTERS")

In [254]:
symbols = df["Symbol"].tolist()
symbols = list(map(lambda x: x.replace('.', '\\.'), symbols))

corporations = df["Corp"].tolist()
corporations = list(map(lambda x: x.replace('.', '\\.'), corporations))

In [302]:
def has_intersection(first, second):
    if first[0] < second[0]:
        if first[1] < second[0] :
            return False
        else :
            return True
    else:
        if first[0] > second[1] :
            return False
        else :
            return True

def combine(first, second, text):
    start = min(first['span'][0],second['span'][0])
    end = max(first['span'][1],second['span'][1])
    new_span =  (start,end)
    new_marker = text[start:end]
    return new_span,new_marker

def remove_type_overlap(match_list, text):
    n = len(match_list)
    if n == 0:
      return []
    i = 0
    j = 1
    while (i < n and j < n):
        first = match_list[i]
        second = match_list[j]
        if has_intersection(first['span'],second['span']):
            new_span, new_marker= combine(first, second, text)
            match_list[i]['span'] = new_span
            match_list[i]['marker'] = new_marker
            match_list[j] = None
        else : 
            i = j
        j+=1    
    
    match_list = list(filter(lambda x: not x is None, match_list))
    return match_list
    
def remove_overlap(OUTPUT, text):
    for key in OUTPUT.keys():
        OUTPUT[key] = remove_type_overlap(OUTPUT[key], text)
    return OUTPUT

def print_output(OUTPUT, text):
    for key in OUTPUT.keys():
        for o in OUTPUT[key]:
          print('----')
          print(text[o['span'][0]:o['span'][1]])
          print(o)


In [305]:
import re
from spacy.tokens import Span
from spacy import displacy

def convert_subtree_to_str(token, remove_ADP=False):
    extracted_event = ''
    start = None
    end = None
    for x in token.subtree:
        if x.dep_ == 'case':
          continue
        if x.pos_ == 'ADV':
          continue
        if remove_ADP and x.pos_ == 'ADP':
            if x.head.text == token.text:
                if not start is None:
                  print('WARNING ------------------> check the convert subtree to str')
                continue

        extracted_event += str(x) + ' '
        if start is None:
            start = x.idx
        end = x.idx + len(x.text)
        
    extracted_event = extracted_event[:-1]
    return extracted_event, start, end

TRANSLATE = {
    SYMBOL: 'نماد',
    CORP: 'شرکت',
    EVENT: 'واقعه',
    ANNOUNCE:'اعلان',
    ANALYSE:'تحلیل',
    CHARACTERS:'شخصیت'
}



def create_output(OUTPUT, output_type, marker, span, **kwargs):
    print(f'span :{span} , text: {text[span[0]:span[1]]} ')
    persian_output_type = TRANSLATE[output_type]
    defaults = {
            "type":persian_output_type, 
            "marker":marker, 
            "span":span,
            }
    OUTPUT[output_type].append(
        {
            **defaults,
            **kwargs
        })
    
    return {
        **defaults,
        **kwargs
    }


def create_spans(label):

    term_spans = list(map(
        lambda match: Span(doc, match[1], match[2], label=stanza_nlp.vocab.strings[match[0]]),
        matcher(doc)
        ))
    
    term_spans = list(filter(
        lambda span: span is not None,
        term_spans
        ))
    
    return term_spans

def concat_tokens(tokens):
  concat = ''
  start = None
  end = None
  for token in tokens:
      if start is None:
        start = token.idx
      substring, s, e = convert_subtree_to_str(token)
      end = e
      concat += substring + ' '
  concat = concat[:-1]
  return concat, start, end 

def get_root_string(main_noun):
  left_childs = list(filter(lambda x: not x.dep_ in ['cop','nsubj'], main_noun.lefts))
  right_childs = list(filter(lambda x: not x.dep_ in ['cop','nsubj'], main_noun.rights))
  extracted_event = ''
  start = None
  end = None
  if left_childs:
    substring, s, e = concat_tokens(left_childs)
    if start is None:
      start = s
    end = e
    extracted_event += substring
  extracted_event += ' ' + main_noun.text + ' '
  if start is None:
    start = main_noun.idx
  end = main_noun.idx + len(main_noun.text)

  if right_childs:
    substring, s, e = concat_tokens(right_childs)
    extracted_event += substring
    end = e
  return extracted_event, start, end


symbols_expression = "|".join(symbols)
corporation_expression = "|".join(corporations)

for text_index, text in enumerate(texts):
    OUTPUT = {
      SYMBOL: [],
      CORP: [],
      EVENT: [],
      ANNOUNCE: [],
      ANALYSE: [],
      CHARACTERS:[]
    }

    text = normalizer1.normalize(text)
    text = normalizer2.normalize(text)

    print(f'Text {text_index + 1}...')
    print(f"Normalized Text: {text}")

    doc = stanza_nlp(text)
    # senetences = doc.sents
    
    symbol_spans = list(map(
        lambda match: doc.char_span(*match.span(), label=SYMBOL),
        re.finditer(symbols_expression, text)
        ))
    
    symbol_spans = list(filter(
        lambda span: span is not None,
        symbol_spans
        ))
    
    corporation_spans = list(map(
        lambda match: doc.char_span(*match.span(), label=CORP),
        re.finditer(corporation_expression, text)
        ))
    
    corporation_spans = list(filter(
        lambda span: span is not None,
        corporation_spans
        ))
    

    term_spans = list(map(
        lambda match: Span(doc, match[1], match[2], label=stanza_nlp.vocab.strings[match[0]]),
        matcher(doc)
        ))
    
    term_spans = list(filter(
        lambda span: span is not None,
        term_spans
        ))

   
    spans = symbol_spans + corporation_spans + term_spans
    doc.set_ents(spans)

    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in symbol_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in corporation_spans:
            retokenizer.merge(span, attrs)

    with doc.retokenize() as retokenizer:
        attrs = {'POS': "NOUN"}
        for span in term_spans:
            retokenizer.merge(span, attrs)

    symbol_ents = list(filter(lambda ent: ent.label_ == SYMBOL,
                     doc.ents))
    
    for symbol_ent in symbol_ents:
        token = symbol_ent[0]
        start = token.idx
        end = token.idx + len(token.text)
        print(create_output(OUTPUT, SYMBOL, token.text, (start,end)))

    corporation_ents = list(filter(lambda ent: ent.label_ == CORP,
                     doc.ents))
    
    for corporation_ent in corporation_ents:
        token = corporation_ent[0]
        start = token.idx
        end = token.idx + len(token.text)
        print(create_output(OUTPUT, CORP, token.text, (start,end)))

    
    term_ents = list(filter(lambda ent: ent.label_ not in [SYMBOL, CORP],
                     doc.ents))

    from spacy.symbols import VERB, NOUN, AUX
    
    for term_ent in term_ents:
        token = term_ent[0]
        # print('------------')
        # print('token is: ', token.text)
        
        if token.dep_ == 'compound:lvc':
            # print('type is: coumpound:lvd')
            # print('parent is: ', token.head)
            if token.head.pos == VERB:
                # print('parent was a verb')
                # print('children of parent: ', list(token.head.children))
                subject = None
                for child in token.head.children:
                    if child.dep_ == 'nsubj':
                        # print('Found a subject, outputing subtree of subject as subject')
                        subject, start_subj, end_subj = convert_subtree_to_str(child)
                        break
                    else:
                        print('NOT IMP - child dep is: ', child.dep_)
                        pass
                else:
                  print('didn\'t found any subj')
                  pass

                token_string, start, end = convert_subtree_to_str(token)
                end = token.head.idx + len(token.head.text)
                extracted_event = token_string + ' ' + token.head.text 
                if subject:
                    print(create_output(OUTPUT, token.ent_type_, extracted_event, (start,end), subject=subject, span_subject=(start_subj,end_subj)))
                else:
                    print(create_output(OUTPUT, token.ent_type_, extracted_event, (start,end)))
            else:
                extracted_event, start, end = convert_subtree_to_str(token)
                print(create_output(OUTPUT, token.ent_type_, extracted_event, (start,end)))


        elif token.dep_ == 'nmod' or token.dep_ == 'amod':
            # print('type is: ', token.dep_)
            # print('parent is: ', token.head)

            # print('finding parent that is not nmod or amod')
            main_noun = token
            ## print('1', main_noun, main_noun.dep_, main_noun.head.pos_)
            while (main_noun.dep_ == 'nmod' or main_noun.dep_ == 'amod') and main_noun.head.pos == NOUN:
                main_noun = main_noun.head
                ## print('2', main_noun, main_noun.dep_, main_noun.head.pos_)
            if main_noun.pos != NOUN:
                print('WARNING ------------------> check code in else: dep = nmod amod')
                continue

            # print('Found a Noun parent that is not nmod or amod')

            if main_noun.dep_ == 'root':
                  # print('Noun parent is root, text: ', main_noun.text)
                  children = list(main_noun.children)
                  # print('children are: ', children)
                  # print('left chilren are: ', list(main_noun.lefts))
                  # print('right chilren are: ', list(main_noun.rights))
                  for child in children:
                    # print('child text: ', child.text, ' pos: ', child.pos_, ' dep_: ', child.dep_)
                    if child.dep_ == 'cop' and child.pos == AUX:
                      extracted_event, start, end = get_root_string(main_noun)
                      print(create_output(OUTPUT, token.ent_type_, extracted_event, (start, end)))
                      break
                  else:
                    #TODO: this is of no use
                    extracted_event, start, end = get_root_string(main_noun)
                    print(create_output(OUTPUT, token.ent_type_, extracted_event, (start, end)))

            else:
                extracted_event, start, end = convert_subtree_to_str(main_noun, remove_ADP=True)
                # extracted_event, start, end = convert_subtree_to_str(main_noun, not_in=['ADP'])

                print(create_output(OUTPUT, token.ent_type_, extracted_event, (start,end)))

        elif token.dep_ == 'root':
            extracted_event, start, end = get_root_string(token)
            print(create_output(OUTPUT, token.ent_type_, extracted_event, (start, end)))

        else:
            # print('type is: other')
            extracted_event, start, end = convert_subtree_to_str(token, remove_ADP=True)

            # print(text[start:end])
            print(create_output(OUTPUT, token.ent_type_, extracted_event, (start,end)))
    # print('Entities: ...')
    displacy.render(doc, style='ent', jupyter=True)
    
    # print('Dependency Tree: ...')
    displacy.render(doc, style='dep', jupyter=True)

    OUTPUT = remove_overlap(OUTPUT, text)
    print_output(OUTPUT, text)





Text 1...
Normalized Text: جلسه‌ای برای افزایش کارایی و افزایش نقد شوندگی بازار سرمایه مقرر شد
span :(13, 59) , text: افزایش کارایی و افزایش نقد شوندگی بازار سرمایه 
{'type': 'واقعه', 'marker': 'افزایش کارایی و افزایش نقد شوندگی بازار سرمایه', 'span': (13, 59)}
span :(27, 59) , text: و افزایش نقد شوندگی بازار سرمایه 
{'type': 'واقعه', 'marker': 'و افزایش نقد شوندگی بازار سرمایه', 'span': (27, 59)}


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


----
افزایش کارایی و افزایش نقد شوندگی بازار سرمایه
{'type': 'واقعه', 'marker': 'افزایش کارایی و افزایش نقد شوندگی بازار سرمایه', 'span': (13, 59)}
Text 2...
Normalized Text: رشد مثبت بازار خوب است
span :(0, 14) , text: رشد مثبت بازار 
{'type': 'واقعه', 'marker': 'رشد مثبت بازار', 'span': (0, 14)}
span :(0, 14) , text: رشد مثبت بازار 
{'type': 'واقعه', 'marker': 'رشد مثبت بازار', 'span': (0, 14)}


[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


----
رشد مثبت بازار
{'type': 'واقعه', 'marker': 'رشد مثبت بازار', 'span': (0, 14)}
